# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope(y_axis):
    global data
    ma_len = len(y_axis)
    
    x_axis = []
    for i in range(ma_len):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global data
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    tick_avg = []
    spread_avg = []
    tick_sd = []
    #tick_act = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])
        
        #tick_act.append(tick_list[-1])
        tick_avg.append(np.mean(tick_list))
        spread_avg.append(np.mean(spread_list))
        tick_sd.append(np.std(tick_list))
        
    #temp_df['tick_act'] = tick_act      
    temp_df['tick_avg'] = tick_avg  
    temp_df['spread_avg'] = spread_avg  
    temp_df['tick_sd'] = tick_sd  
    
    return(temp_df)

In [4]:
def before_sma():
    global data    
    data['ssma_list'].append(val)    
    return()

def after_sma():
    global data
    
    data['ssma_list'].popleft()
    data['ssma_list'].append(val)
    data['sema'] = list(pd.DataFrame(list(data['ssma_list'])).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]
    
    if len(data['sema_ready']) < 2:
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = np.nan

    elif len(data['sema_ready']) > 1:
        data['sema_ready'].popleft()
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = data['sema_ready'][-1] - data['sema_ready'][len(data['sema_ready'])-2]
    
    return()

def before_lma():
    global data    
    data['lsma_list'].append(val)    
    return()

def after_lma():
    global data
    
    data['lsma_list'].popleft()
    data['lsma_list'].append(val)
    data['lema'] = list(pd.DataFrame(list(data['lsma_list'])).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]
    
    if len(data['lema_ready']) < 2:
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = np.nan

    elif len(data['lema_ready']) > 1:
        data['lema_ready'].popleft()
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = data['lema_ready'][-1] - data['lema_ready'][len(data['lema_ready'])-2]
    
    return()

In [5]:
def roll_ma(ma_list):
    global data
    ma_len = len(ma_list)
    sema_val = list(pd.DataFrame(ma_list).ewm(span=ma_len).mean()[0])[ma_len - 1]    
    return(sema_val)

## File paths

#### 1. High number of ticks ensures pip level movement and pip level predictions
#### 2. High sma_len helps ml model prediction accuracy (Predictable curve)
#### 3. High sma_len remove connection between actual tick_avg and sema (Prediction is high, but actual prediction is poor)

In [6]:
year = 2020

data = {}
data['number_of_ticks'] = 300
data['rsi_window'] = 14
data['sma_len'] = 5
data['lma_len'] = 10

diff_col = 'sema'
data['pip_diff'] = 0.0001

source_file_path = f'data/yearly_tick_data/{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/yearly_tick_data/2020.csv
chunk_file_path : data/yearly_tick_data\chunk_2020.csv
target_file_path : data/yearly_tick_data\tab_2020.csv


## Read data

In [7]:
%%time
df = pd.read_csv(source_file_path, nrows=100000)
#df = pd.read_csv(source_file_path)
df.head()

Wall time: 187 ms


,DateTime,Bid,Ask,Volume
0,20200101 22:01:12.821,1.12106,1.12160,1
1,20200101 22:01:17.176,1.12120,1.12160,1
2,20200101 22:01:18.545,1.12117,1.12160,1
3,20200101 22:01:19.145,1.12123,1.12161,1
4,20200101 22:01:19.246,1.12120,1.12161,1


## Data manipulation

In [8]:
%%time

df = chunk_ticks(df, data['number_of_ticks'])
df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')

df = pd.read_csv(chunk_file_path)
df.head()

100%|██████████████████████████████████████████████████████████████████████████████| 556/556 [00:00<00:00, 3384.37it/s]


Records : 556
Wall time: 202 ms


,tick_avg,spread_avg,tick_sd
0,1.121427,0.000314,0.000066
1,1.121469,0.000125,0.000048
2,1.121558,0.000166,0.000019
3,1.121609,0.000109,0.000016
4,1.121622,0.000119,0.000012


In [9]:
%%time

data['rs_max'] = 1e6

data['ssma_list'] = collections.deque([])
data['lsma_list'] = collections.deque([])
data['sema_ready'] = collections.deque([])
data['lema_ready'] = collections.deque([])
df['sema'] = ''
df['lema'] = ''
df['sema_diff'] = ''
df['lema_diff'] = ''


# RSI -----------------------------
df['diff'] = df['tick_avg'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Simple Moving Averages ------------------
df['ssma'] = df['tick_avg'].rolling(window=data['sma_len']).mean()
df['ssma_diff'] = df['ssma'].diff()
df['lsma'] = df['tick_avg'].rolling(window=data['lma_len']).mean()
df['lsma_diff'] = df['lsma'].diff()
df['sma_diff'] = df['ssma'] - df['lsma']

df['max_tick'] = df['tick_avg'].rolling(window=data['sma_len']).max()
df['min_tick'] = df['tick_avg'].rolling(window=data['sma_len']).min()

df['max_gap'] = df['max_tick'] -  df['tick_avg']
df['min_gap'] = df['min_tick'] - df['tick_avg']

Wall time: 18.9 ms


In [10]:
%%time
# Emas ----------------
df['sema'] = df['tick_avg'].rolling(window=data['sma_len']).progress_apply(roll_ma)
df['lema'] = df['tick_avg'].rolling(window=data['lma_len']).progress_apply(roll_ma)

df['sema_diff'] = df['sema'].diff()
df['lema_diff'] = df['lema'].diff()

552it [00:00, 1079.51it/s]
547it [00:00, 1077.84it/s]

Wall time: 1.04 s


In [11]:
%%time
# Slopes -----------------------------
df['small_sema_slope'] = df['sema'].rolling(window=data['sma_len']).progress_apply(get_slope)
df['long_sema_slope'] = df['sema'].rolling(window=data['lma_len']).progress_apply(get_slope)

548it [00:00, 3390.26it/s]
543it [00:00, 3699.73it/s]

Wall time: 311 ms


In [12]:
%%time

df['ema_diff'] = df['sema'] - df['lema']

# Direction -------------------------
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

# Remove NaNs ------------------------
del df['gain']
del df['loss']
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

df.tail()

Total records : 543
Wall time: 6.98 ms


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,small_sema_slope,long_sema_slope,ema_diff,direction
538,1.116332,0.000020,0.000039,1.116488,1.116644,-0.000125,-0.000088,-0.000056,0.000033,0.000056,...,-0.000054,-0.000133,1.117085,1.116332,0.000753,0.0,-85.488309,-73.369814,-0.000155,decrease
539,1.116259,0.000018,0.000077,1.116370,1.116556,-0.000118,-0.000088,-0.000073,0.000026,0.000062,...,-0.000063,-0.000235,1.116805,1.116259,0.000547,0.0,-85.794371,-79.634945,-0.000186,decrease
540,1.116093,0.000023,0.000048,1.116242,1.116449,-0.000128,-0.000107,-0.000165,0.000024,0.000073,...,-0.000080,-0.000298,1.116517,1.116093,0.000423,0.0,-85.555723,-82.478797,-0.000208,decrease
541,1.115939,0.000024,0.000160,1.116112,1.116331,-0.000130,-0.000119,-0.000155,0.000018,0.000084,...,-0.000092,-0.000322,1.116387,1.115939,0.000449,0.0,-85.427303,-84.081884,-0.000219,decrease
542,1.115757,0.000019,0.000078,1.115962,1.116195,-0.000150,-0.000136,-0.000181,0.000018,0.000097,...,-0.000112,-0.000336,1.116332,1.115757,0.000574,0.0,-85.642708,-85.073989,-0.000233,same


## Write data to csv

In [13]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 94.8 ms


## Print Report

In [14]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head(5)

          counts  percentage
same         450   82.872928
decrease      64   11.786372
increase      29    5.340700


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,small_sema_slope,long_sema_slope,ema_diff,direction
0,1.122073,0.000031,0.000096,1.122037,1.121934,0.000037,0.000047,0.000029,0.000053,0.000007,...,0.000046,0.000179,1.122073,1.121969,0.000000,-0.000105,80.015591,80.169438,0.000103,same
1,1.121848,0.000034,0.000040,1.121968,1.121925,-0.000069,-0.000009,-0.000226,0.000053,0.000023,...,0.000023,0.000132,1.122073,1.121848,0.000226,0.000000,66.982939,79.585603,0.000043,same
2,1.121867,0.000031,0.000030,1.121926,1.121921,-0.000041,-0.000003,0.000019,0.000051,0.000023,...,0.000026,0.000076,1.122073,1.121848,0.000207,-0.000019,-33.789900,77.607538,0.000005,same
3,1.121825,0.000027,0.000042,1.121885,1.121909,-0.000042,-0.000013,-0.000041,0.000045,0.000026,...,0.000017,0.000026,1.122073,1.121825,0.000248,0.000000,-73.679140,72.716979,-0.000024,same
4,1.121661,0.000030,0.000038,1.121791,1.121866,-0.000094,-0.000043,-0.000164,0.000041,0.000038,...,0.000006,-0.000057,1.122073,1.121661,0.000412,0.000000,-80.152921,45.254197,-0.000075,same


In [15]:
# Direction -------------------------
#diff_col = 'tick_act'
diff_col = 'tick_avg'

df['act_direction'] = 'same'
df['act_direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['act_direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

print('prediction : same')
print(df.loc[df['direction'] == 'same', 'act_direction'].value_counts(normalize=True))
print('-------------')

print('prediction : increase')
print(df.loc[df['direction'] == 'increase', 'act_direction'].value_counts(normalize=True))
print('-------------')

print('prediction : decrease')
print(df.loc[df['direction'] == 'decrease', 'act_direction'].value_counts(normalize=True))
print('-------------')

print('\n')
print(df[['tick_avg', 'sema', 'direction','act_direction']].head(50))
del df['act_direction']

prediction : same
same        0.706667
increase    0.153333
decrease    0.140000
Name: act_direction, dtype: float64
-------------
prediction : increase
increase    0.896552
same        0.103448
Name: act_direction, dtype: float64
-------------
prediction : decrease
decrease    0.78125
same        0.21875
Name: act_direction, dtype: float64
-------------


    tick_avg      sema direction act_direction
0   1.122073  1.122037      same      decrease
1   1.121848  1.121968      same          same
2   1.121867  1.121926      same          same
3   1.121825  1.121885      same      decrease
4   1.121661  1.121791      same          same
5   1.121629  1.121714      same          same
6   1.121633  1.121677      same          same
7   1.121654  1.121658      same      increase
8   1.121829  1.121715      same          same
9   1.121848  1.121769  increase      increase
10  1.122070  1.121892      same          same
11  1.122094  1.121982  increase      increase
12  1.122263  1.122107  increa